In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

/Users/hasan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
data = pd.read_csv('Classe_A_per_nome_commerciale_16_12_2024.csv', sep=';')
data = data.iloc[5113:5444]
DropList = ['Prezzo al pubblico �','X=in lista di trasparenza Aifa 16-12-2024 ','Solo in lista di Regione:', 'Metri cubi ossigeno ']

for col in DropList:
    if col in data.columns:
        data.drop(col, axis=1, inplace=True)


In [4]:
data.columns

Index(['Principio Attivo', 'Descrizione Gruppo', 'Denominazione e Confezione',
       'Titolare AIC', 'AIC', 'Codice Gruppo Equivalenza'],
      dtype='object')

In [5]:
data.head()

,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,AIC,Codice Gruppo Equivalenza
5113,Levotiroxina,LEVOTIROXINA 112MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 112 mcg,IBSA FARMACEUTICI ITALIA Srl,46860072,LPJ
5114,Levotiroxina,LEVOTIROXINA 125MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 125 mcg,IBSA FARMACEUTICI ITALIA Srl,46860084,LPK
5115,Levotiroxina,LEVOTIROXINA 13MCG/ML 1ML 30 UNITA' USO ORALE ...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 13 mcg,IBSA FARMACEUTICI ITALIA Srl,46860019,LPH
5116,Levotiroxina,LEVOTIROXINA 137MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 137 mcg,IBSA FARMACEUTICI ITALIA Srl,46860096,LPL
5117,Levotiroxina,LEVOTIROXINA 150MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 150 mcg,IBSA FARMACEUTICI ITALIA Srl,46860108,LPM


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 5113 to 5443
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Principio Attivo            331 non-null    object
 1   Descrizione Gruppo          331 non-null    object
 2   Denominazione e Confezione  331 non-null    object
 3   Titolare AIC                331 non-null    object
 4   AIC                         331 non-null    int64 
 5   Codice Gruppo Equivalenza   331 non-null    object
dtypes: int64(1), object(5)
memory usage: 15.6+ KB


In [5]:
data['AIC'] = data['AIC'].astype(str).apply(lambda x: '0' + x)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 5113 to 5443
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Principio Attivo            331 non-null    object
 1   Descrizione Gruppo          331 non-null    object
 2   Denominazione e Confezione  331 non-null    object
 3   Titolare AIC                331 non-null    object
 4   AIC                         331 non-null    object
 5   Codice Gruppo Equivalenza   331 non-null    object
dtypes: object(6)
memory usage: 15.6+ KB


In [6]:
data.head()

,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,AIC,Codice Gruppo Equivalenza
5113,Levotiroxina,LEVOTIROXINA 112MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 112 mcg,IBSA FARMACEUTICI ITALIA Srl,046860072,LPJ
5114,Levotiroxina,LEVOTIROXINA 125MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 125 mcg,IBSA FARMACEUTICI ITALIA Srl,046860084,LPK
5115,Levotiroxina,LEVOTIROXINA 13MCG/ML 1ML 30 UNITA' USO ORALE ...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 13 mcg,IBSA FARMACEUTICI ITALIA Srl,046860019,LPH
5116,Levotiroxina,LEVOTIROXINA 137MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 137 mcg,IBSA FARMACEUTICI ITALIA Srl,046860096,LPL
5117,Levotiroxina,LEVOTIROXINA 150MCG/ML 1ML 30 UNITA' USO ORALE...,LEVOTIRSOL*orale soluz 30 flaconcini 1 ml 150 mcg,IBSA FARMACEUTICI ITALIA Srl,046860108,LPM


In [7]:
data.to_csv('fixeddata.csv')

In [10]:
WAIT = 3

df = pd.read_csv("fixeddata.csv")
aic_list = df["AIC"].astype(str).str.zfill(9).tolist()

results = []

for aic_code in aic_list:
    print(f"\n🔎 Searching AIC code: {aic_code}")

    driver = webdriver.Safari()
    driver.get("https://medicinali.aifa.gov.it/it/#/it/")
    time.sleep(WAIT)

    try:
        checkbox = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='disclaimercheck']"))
        )
        checkbox.click()
        print("✅ Checkbox marked.")
        time.sleep(1)
    except:
        print("⚠️ Checkbox doesn't exist.")

    try:
        confirm_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH,
                "//button[contains(text(), \"Ho letto e compreso\") or contains(text(), 'ACCETTA') or contains(text(), 'Conferma')]"
            ))
        )
        confirm_button.click()
        print("✅ Chechbox marked.")
        time.sleep(1)
    except:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH,
                    "//*[@id='acceptAllCookies' or contains(text(), 'Accetta')]"
                ))
            )
            accept_button.click()
            print("✅ Cookies marked.")
            time.sleep(1)
        except:
            try:
                driver.execute_script("document.getElementById('acceptAllCookies')?.click();")
                print("✅ Cookies marked.")
                time.sleep(1)
            except:
                print("⚠️ Cookies not marked.")

    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='text']"))
        )
        search_box.clear()
        search_box.send_keys(aic_code)
        search_box.send_keys(Keys.RETURN)
        time.sleep(WAIT)

        result_xpath = '//*[@id="main-content"]/app-content/div/div/app-results-page/div[3]/div[2]/div[2]/div/div/app-forma-dosaggio/a/div/div[1]/span'
        link = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, result_xpath))
        )
        link.click()
        time.sleep(WAIT)

        try:
            info_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH,
                    '//*[@id="main-content"]/app-content/div/div/app-details-page/div/div[1]/div[2]/div[1]/div/div[2]/div[3]/ul/li'
                ))
            )
            info_text = info_element.text.strip()
            print(f"ℹ️ Additional Info: {info_text}")
        except:
            info_text = None
            print("⚠️ Additional Info not found.")

        rcp_links = driver.find_elements(By.XPATH,
            '//*[@id="main-content"]/app-content/div/div/app-details-page/div/div[1]/div[2]/div[1]/div/div[1]/div[2]/button'
        )

        if rcp_links:
            rcp_links[0].click()
            time.sleep(WAIT)
            driver.switch_to.window(driver.window_handles[-1])
            pdf_url = driver.current_url
            print(f"✅ AIC {aic_code} for RCP PDF: {pdf_url}")
        else:
            pdf_url = None
            print("⚠️ RCP connection not found.")

        results.append((aic_code, pdf_url, info_text))

    except Exception as e:
        print(f"❌ Error: {e}")
        results.append((aic_code, None, None))

    driver.quit()
result_df = pd.DataFrame(results, columns=["AIC Code", "RCP PDF URL", "Info Text"])
result_df.to_csv("rcp_results.csv", index=False, encoding='utf-8')

print("\n📄 All results are saved in file 'rcp_results.csv'.")


🔎 Searching AIC code: 046860072


WebDriverException: Message: 


In [1]:
# CSV dosyalarını oku
df1 = pd.read_csv("fixeddata.csv")
df2 = pd.read_csv("rcp_results.csv")

birlesik_df = pd.concat([df1, df2], axis=1)

# Sonucu kaydet
birlesik_df.to_csv("merged.csv", index=False)

NameError: name 'pd' is not defined

In [2]:
df = pd.read_csv("merged.csv")
df[df["RCP PDF URL"].isna()]

NameError: name 'pd' is not defined

In [ ]:
df = pd.read_csv("merged.csv")
df[df["RCP PDF URL"].isna()]

In [ ]:
df = pd.read_csv("merged.csv")
temiz_df = df.dropna()

temiz_df.to_csv("clear.csv", index=False)

df = pd.read_csv("clear.csv")
df